In [1]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

load_dotenv()

/Users/shivangraikar/opt/anaconda3/envs/rag/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:

try:
    pc = Pinecone(api_key = os.getenv('PINECONE_API_KEY'))
    pc.create_index(
        name = 'rag',
        dimension = 384,
        metric = 'cosine',
        spec = ServerlessSpec(
            cloud = 'aws',
            region = 'us-east-1'
        )
    )
except:
    pass 

In [3]:
import json 
data = json.load(open('reviews.json'))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Great professor, explains concepts clearly and is very approachable.'},
 {'professor': 'Prof. Emily Johnson',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Amazing teacher! Her lectures are well-organized and easy to follow.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Physics',
  'stars': 3,
  'review': 'The class was tough, but he knows his stuff. Be prepared to study hard.'},
 {'professor': 'Prof. Sarah Lee',
  'subject': 'Biology',
  'stars': 2,
  'review': 'Lectures can be boring, and the material is not well-organized.'},
 {'professor': 'Dr. David Wilson',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Very enthusiastic about the subject and makes the lectures interesting.'},
 {'professor': 'Prof. Linda Davis',
  'subject': 'History',
  'stars': 4,
  'review': 'Good professor, but the exams are a bit challenging.'},
 {'professor': 'Dr. Karen Martinez',
  'subject': 

In [7]:
from sentence_transformers import SentenceTransformer

processed_data = [] 
for review in data['reviews']:
    model = SentenceTransformer("all-MiniLM-L6-v2")
    sentence = review['review']
    
    embeddings = model.encode(sentence)
    print(embeddings.shape)
    processed_data.append( {
        'values': embeddings,
        'id': review['professor'],
        'metadata': {
            'review': review['review'],
             'subject': review['subject'],
             'stars' : review['stars']  
        }
    })

(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)


In [8]:
processed_data[0]

{'values': array([ 1.06635662e-02, -2.35913657e-02, -4.57718410e-02, -3.75261381e-02,
        -7.55031407e-02, -3.62167545e-02,  3.27636139e-03,  4.99032475e-02,
        -7.78784454e-02,  3.29818577e-02, -2.75560413e-02,  8.09926987e-02,
        -6.62412420e-02,  5.87010346e-02, -1.46907140e-02, -1.62285368e-03,
        -4.80473042e-03, -8.72953609e-02, -1.24082621e-02, -1.38314888e-01,
         8.16781372e-02,  3.91527377e-02,  4.39176522e-02, -3.33522223e-02,
        -7.35988002e-03,  2.62389686e-02,  1.29018240e-02, -1.61386915e-02,
         9.27816927e-02, -9.37808398e-03, -4.97294590e-02,  4.84152399e-02,
        -7.28527829e-03,  1.11110210e-02, -5.94621003e-02,  7.92611316e-02,
        -3.85853201e-02,  1.30568683e-01,  1.14613390e-02, -9.18410346e-03,
        -7.67827109e-02,  3.92479710e-02,  3.07301041e-02, -2.81365532e-02,
         3.41194309e-03, -6.30579293e-02,  2.46548355e-02, -3.53734829e-02,
        -1.86381098e-02, -5.24565168e-02, -9.74889770e-02, -1.58188567e-02,
  

In [9]:
index = pc.Index('rag')
index.upsert(
    vectors = processed_data,
    namespace = "ns1"
)

{'upserted_count': 20}

In [10]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}